In [ ]:

import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

running_dir = r"/Users/4JStudent/Desktop/Code/Bens_Tetris_Regression"

In [ ]:
## Dataloading
df = pd.read_csv (os.path.join(running_dir, "sprint_data.csv"))
df.columns = df.columns.str.lower()
df["normalized_pps"] = df["approxpps"].map (lambda x: 1 / (1 + np.exp(-x)))

df.head()

In [ ]:
import torch
import torch.nn as nn
class RegressionModel (nn.Module):
  def __init__ (self, param_names: list):
    super ().__init__()
    self.initialize_weights(len(param_names))
    self.layer = nn.Linear (1, 1)

    # Paramaters
    exec (
      ", ".join(f"self.{name}" for name in param_names) + ' = ' + 
      ", ".join(
        [
          f'nn.Parameter(self.weights[{i}])' 
          for i in range (len(param_names))
          ]
      )
    )

  # Weights
  def initialize_weights (self, n):
    self.weights = [nn.init.uniform (torch.empty (1)) for i in range (n)]

  # Function
  def forward (self, x):
    return self.a * x + self.b
    # return self.layer (x)

    

In [ ]:

params = ["a", "b", "c"]
model = RegressionModel (params)

optimizer = torch.optim.Adam (model.parameters(), lr=0.2)
criterion = nn.L1Loss()

def plot_regression (df, model, min_scale=0, max_scale=10):
  sns.set_theme()
  sns.scatterplot (df, x="ln(days)", y="ln(pps)", s=5)

  xModel = torch.linspace(min_scale, max_scale, steps=500)
  yModel = model(xModel)
  # print (yModel)

  plt.plot(xModel.detach(), yModel.detach())
  plt.show()
  plt.close('all')

def train_regression (model, data, epochs=1, optimizer=None, criterion=None):
    for i in tqdm(range (epochs)):
      x, y = torch.unsqueeze(data[0], 1), torch.unsqueeze(data[1], 1)
      # for x, y in zip(x_data, y_data):      
        # print (f"{x.shape=} {y.shape=}")
        # print (f"{x=} {y=}")
      y_pred = model(x.type (torch.DoubleTensor))
      loss = criterion(y_pred, y)
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()


xData, yData = torch.tensor(df["ln(days)"]), torch.tensor(df["ln(pps)"])
plot_regression (df, model, min(xData), max(xData))
train_regression (model, (xData, yData), epochs=40, optimizer=optimizer, criterion=criterion)
plot_regression (df, model, min(xData), max(xData))
